In [ ]:
# IMPORT PACKAGES
import os
from ultralytics import YOLO

In [ ]:
# === USER CONFIGURATION ===
SELECTED_CV_TYPE = "sgkf05"                 # Choose between 'sgkf05'
MODEL_NAME = "yolo11s-cls"                  # Choose between YOLO models such as 'yolo11n-cls' or 'yolo11s-cls'
MODEL_CONFIG_PATH = MODEL_NAME + ".yaml"    # Path to the model configuration file
MODEL_WEIGHTS = MODEL_NAME + ".pt"          # Path to the model weights file
MODEL_NAME = os.path.splitext(os.path.basename(MODEL_CONFIG_PATH))[0]  # e.g. 'yolo11n-cls'
PROJECT_NAME = f"{SELECTED_CV_TYPE}-{MODEL_NAME.split('-')[0]}"  # Project name for saving results
AUGMENTATION_MODE = "manual"  # Choose 'manual' or 'auto'

NR_OF_EPOCHS = 100  # Number of epochs for training

In [ ]:
# Dynamically detect all data splits from the selected CV type
IMAGE_SIZE = (224, 224)  # Should match preprocessing and dataset split
data_split_base = os.path.join("data", f"2-splits")
data_split_configs = []
config_path = os.path.join(data_split_base, SELECTED_CV_TYPE)
if os.path.isdir(config_path):
    for repeat in sorted(os.listdir(config_path)):
        repeat_path = os.path.join(config_path, repeat)
        if os.path.isdir(repeat_path):
            for fold in sorted(os.listdir(repeat_path)):
                fold_path = os.path.join(repeat_path, fold)
                if os.path.isdir(fold_path):
                    data_split_configs.append((SELECTED_CV_TYPE, repeat, fold, fold_path))

In [ ]:
# TRAINING CONFIGURATION
TRAIN_CONFIG = dict(
    # For reference go to: https://docs.ultralytics.com/modes/train/#train-settings

    # TRAINING SETTINGS
    patience=20,
    batch=32,
    workers=8,
    exist_ok=True,
    optimizer="AdamW",
    seed=0,
    single_cls=True,
    cos_lr=True,
    lr0=0.0001,
    lrf=0.1,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    dropout=0.4,
    label_smoothing=0.1
)

# Apply augmentation settings based on the mode
if AUGMENTATION_MODE == "auto":
    TRAIN_CONFIG["auto_augment"] = "autoaugment"
else:
    TRAIN_CONFIG.update(dict(
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=2,
        translate=0.4,
        scale=0.5,
        shear=10,
        perspective=0,
        flipud=0.0,
        fliplr=0.5,
        bgr=0.0,
        mosaic=0.0,
        mixup=0.4,
        copy_paste=0.0,
        erasing=0,
        crop_fraction=1.0,
    ))

In [ ]:
def delete_model_files():
    for file in os.listdir():
        if file.endswith(".pt"):
            os.remove(file)

In [ ]:
def train_model(cv_type, repeat_name, fold_name, data_path):    
    try:
        # Delete old models
        delete_model_files()

        # Load the model
        model = YOLO(MODEL_CONFIG_PATH).load(MODEL_WEIGHTS)

        TRAIN_CONFIG["project"] = os.path.join("runs", "classify", PROJECT_NAME)        
        TRAIN_CONFIG["name"] = f"train-{repeat_name}-{fold_name}"

        results = model.train(data=data_path, 
                              epochs=NR_OF_EPOCHS, imgsz=IMAGE_SIZE[0], 
                              **TRAIN_CONFIG)

        return results
    
    except Exception as e:
        print(f"Error during training on {data_path}: {e}")

In [ ]:
for cv_type, seed, fold, data_split_path in data_split_configs:
    print(f"Training on {data_split_path}")
    print("---------------------------------------------------")
    results=train_model(cv_type, seed, fold, data_split_path)
    print(results)
    print("---------------------------------------------------")